# PVDF + 水の溶媒和系の作成デモ

このノートブックでは、PVDF（ポリフッ化ビニリデン）構造に
水分子を充填して溶媒和系を作成する方法を示します。

## 主な機能
1. PVDF構造ファイルの読み込み
2. 密度指定による水分子数の自動計算
3. ランダム配置と重なりチェック
4. 初期構造の保存
5. （オプション）MDシミュレーションでの緩和

## 応用例
- PVDF膜の含水状態のシミュレーション
- PVDF/水界面の反応性研究
- 電解質溶液とPVDFの相互作用

In [ ]:
import numpy as np
import sys
from pathlib import Path
from ase.io import read, write
from ase import units

# プロジェクトのutilsをインポート
sys.path.append(str(Path.cwd().parent / "LiB2_structure_ipynb"))
from utils.solvation_utils import (
    fill_pvdf_with_water,
    estimate_required_molecules,
    fill_box_with_molecules
)

print("✓ インポート完了")

## 1. 設定パラメータ

In [ ]:
CONFIG = {
    # 入力ファイル（reaxFF_H2O_PVDF_Al_shrink.ipynbと同じディレクトリ）
    'pvdf_structure_path': '/home/jovyan/Kaori/MD/LiB_2/structure/PVDF_only_shrunk.cif',
    
    # 水の密度設定
    'water_density_g_cm3': 0.85,  # 0.85 g/cm³ (PVDFの空隙が少ない可能性を考慮)
    
    # 距離パラメータ
    'min_distance': 2.2,  # PVDFのFと水のHの最小距離 (Å)
    
    # 配置パラメータ
    'target_fill_fraction': 0.80,  # 目標の80%配置できればOK
    'max_attempts_per_molecule': 3000,  # PVDFは密なので試行回数を多めに
    
    # 乱数シード（再現性のため）
    'random_seed': 123,
    
    # 出力ディレクトリ
    'output_dir': 'solvation_results',
}

# 出力ディレクトリの作成
output_dir = Path(CONFIG['output_dir'])
output_dir.mkdir(exist_ok=True)

print("設定:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 2. PVDF構造の読み込み

In [ ]:
pvdf_path = CONFIG['pvdf_structure_path']

try:
    pvdf_atoms = read(pvdf_path)
    print(f"✓ PVDF構造を読み込みました: {pvdf_path}")
    print(f"\n構造情報:")
    print(f"  組成: {pvdf_atoms.get_chemical_formula()}")
    print(f"  原子数: {len(pvdf_atoms)}")
    print(f"  セルパラメータ: {pvdf_atoms.cell.cellpar()}")
    print(f"  体積: {pvdf_atoms.get_volume():.2f} Å³")
    
    # C, H, F の数を数える
    symbols = pvdf_atoms.get_chemical_symbols()
    n_c = symbols.count('C')
    n_h = symbols.count('H')
    n_f = symbols.count('F')
    print(f"\n原子数内訳:")
    print(f"  C: {n_c}")
    print(f"  H: {n_h}")
    print(f"  F: {n_f}")
    print(f"  → PVDFモノマー数 (CH₂CF₂): 約 {min(n_c//2, n_h, n_f//2)}")
    
except FileNotFoundError:
    print(f"✗ エラー: ファイルが見つかりません: {pvdf_path}")
    print(f"\nヒント: 以下のいずれかのファイルを使用してください:")
    print(f"  - PVDF_only_shrunk.cif")
    print(f"  - その他のPVDF構造ファイル")
    raise

## 3. 必要な水分子数の見積もり

In [ ]:
estimate = estimate_required_molecules(
    pvdf_atoms,
    solvent_type='H2O',
    density_g_cm3=CONFIG['water_density_g_cm3']
)

print("水分子数の見積もり:")
print(f"  体積: {estimate['volume_A3']:.2f} Å³ ({estimate['volume_cm3']:.3e} cm³)")
print(f"  目標密度: {estimate['target_density_g_cm3']} g/cm³")
print(f"  水のモル質量: {estimate['solvent_molar_mass']:.3f} g/mol")
print(f"  必要な水分子数: {estimate['n_molecules_required']}")
print(f"  必要な水原子数: {estimate['n_atoms_required']} (H2O x {estimate['n_molecules_required']})")
print(f"\n最終構造予測:")
print(f"  既存原子数: {estimate['existing_atoms']}")
print(f"  追加原子数: {estimate['n_atoms_required']}")
print(f"  総原子数: {estimate['total_atoms']}")

# PVDFは高分子なので密度が高く、水が入る空間が限られる可能性
print(f"\n⚠️ 注意:")
print(f"  PVDFは高分子で密な構造のため、全ての水分子を配置できない可能性があります。")
print(f"  target_fill_fraction = {CONFIG['target_fill_fraction']} に設定しています。")

## 4. 水分子の充填実行

PVDFは密な構造のため、この処理には時間がかかる場合があります。

In [ ]:
solvated_pvdf = fill_pvdf_with_water(
    pvdf_atoms,
    water_density=CONFIG['water_density_g_cm3'],
    min_distance=CONFIG['min_distance'],
    max_attempts_per_molecule=CONFIG['max_attempts_per_molecule'],
    target_fill_fraction=CONFIG['target_fill_fraction'],
    random_seed=CONFIG['random_seed'],
    verbose=True
)

print("\n" + "=" * 60)
print("  充填完了！")
print("=" * 60)
print(f"最終組成: {solvated_pvdf.get_chemical_formula()}")
print(f"総原子数: {len(solvated_pvdf)}")

## 5. 結果の保存

In [ ]:
# ファイル名の生成
density_str = f"{CONFIG['water_density_g_cm3']:.2f}".replace('.', '_')
output_basename = f"pvdf_h2o_d{density_str}"

# XYZ形式で保存（可視化用）
xyz_path = output_dir / f"{output_basename}_initial.xyz"
write(xyz_path, solvated_pvdf)
print(f"✓ XYZ形式で保存: {xyz_path}")

# CIF形式で保存（結晶構造情報を保持）
cif_path = output_dir / f"{output_basename}_initial.cif"
write(cif_path, solvated_pvdf)
print(f"✓ CIF形式で保存: {cif_path}")

# 統計情報をテキストファイルに保存
stats_path = output_dir / f"{output_basename}_stats.txt"
with open(stats_path, 'w') as f:
    f.write("PVDF + H₂O 溶媒和系の統計情報\n")
    f.write("=" * 60 + "\n\n")
    f.write("設定パラメータ:\n")
    for key, value in CONFIG.items():
        f.write(f"  {key}: {value}\n")
    f.write("\n構造情報:\n")
    f.write(f"  組成: {solvated_pvdf.get_chemical_formula()}\n")
    f.write(f"  総原子数: {len(solvated_pvdf)}\n")
    f.write(f"  セル体積: {solvated_pvdf.get_volume():.2f} Å³\n")
    
    # 実際に追加された水分子数を計算
    n_water_atoms = len(solvated_pvdf) - len(pvdf_atoms)
    n_water_molecules = n_water_atoms // 3
    f.write(f"\n追加された水:\n")
    f.write(f"  水分子数: {n_water_molecules}\n")
    f.write(f"  水原子数: {n_water_atoms}\n")
    
    # 実際の密度
    water_mass_g = (n_water_molecules * 18.015) / units.mol
    vol_cm3 = solvated_pvdf.get_volume() * 1e-24
    actual_density = water_mass_g / vol_cm3
    f.write(f"\n実際の密度:\n")
    f.write(f"  水の密度: {actual_density:.3f} g/cm³\n")
    f.write(f"  目標密度: {CONFIG['water_density_g_cm3']} g/cm³\n")
    f.write(f"  達成率: {actual_density/CONFIG['water_density_g_cm3']*100:.1f}%\n")
    
    # 混合比
    pvdf_atoms_count = len(pvdf_atoms)
    water_atoms_count = n_water_atoms
    f.write(f"\n混合比:\n")
    f.write(f"  PVDF原子数: {pvdf_atoms_count}\n")
    f.write(f"  水原子数: {water_atoms_count}\n")
    f.write(f"  水/(PVDF+水): {water_atoms_count/(pvdf_atoms_count+water_atoms_count)*100:.1f}%\n")

print(f"✓ 統計情報を保存: {stats_path}")

print("\n" + "=" * 60)
print("  全ての処理が完了しました！")
print("=" * 60)
print(f"\n出力ファイル:")
print(f"  1. {xyz_path}")
print(f"  2. {cif_path}")
print(f"  3. {stats_path}")

## 6. （オプション）可視化と分析

In [ ]:
# 原子数の内訳
from collections import Counter
symbols = solvated_pvdf.get_chemical_symbols()
counts = Counter(symbols)

print("最終構造の原子数内訳:")
for symbol, count in sorted(counts.items()):
    print(f"  {symbol}: {count}")

# 水分子の分布を確認（Oの座標）
import matplotlib.pyplot as plt

water_indices = [i for i, s in enumerate(symbols) if s == 'O']
if water_indices:
    water_positions = solvated_pvdf.positions[water_indices]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # XY平面
    axes[0].scatter(water_positions[:, 0], water_positions[:, 1], alpha=0.5, s=20)
    axes[0].set_xlabel('X (Å)')
    axes[0].set_ylabel('Y (Å)')
    axes[0].set_title('水分子の分布 (XY平面)')
    axes[0].grid(True, alpha=0.3)
    
    # XZ平面
    axes[1].scatter(water_positions[:, 0], water_positions[:, 2], alpha=0.5, s=20)
    axes[1].set_xlabel('X (Å)')
    axes[1].set_ylabel('Z (Å)')
    axes[1].set_title('水分子の分布 (XZ平面)')
    axes[1].grid(True, alpha=0.3)
    
    # YZ平面
    axes[2].scatter(water_positions[:, 1], water_positions[:, 2], alpha=0.5, s=20)
    axes[2].set_xlabel('Y (Å)')
    axes[2].set_ylabel('Z (Å)')
    axes[2].set_title('水分子の分布 (YZ平面)')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # 保存
    plot_path = output_dir / f"{output_basename}_distribution.png"
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    print(f"\n✓ 分布図を保存: {plot_path}")
    plt.show()
    
    print(f"\n配置された水分子数: {len(water_indices)}")

## 次のステップ

1. **MDシミュレーション**: 作成した構造でMD計算を実行
2. **緩和**: NPT緩和で密度と構造を最適化
3. **反応シミュレーション**: 
   - PVDF-Fと水の相互作用
   - 高温での分解反応
4. **異なる含水率**: `water_density_g_cm3`を変えて比較
5. **界面系**: PVDF/基板/水の3相系の構築

### 推奨設定

**含水率の調整**:
- **低含水**: `water_density_g_cm3 = 0.3-0.5` (わずかに湿潤)
- **中含水**: `water_density_g_cm3 = 0.6-0.8` (部分的に充填)
- **高含水**: `water_density_g_cm3 = 0.9-1.0` (完全に充填)

**距離設定**:
- `min_distance = 2.0-2.5 Å` (PVDFのF原子と水のH原子の距離)

**配置効率**:
- PVDFは密な構造のため、`target_fill_fraction = 0.7-0.8` を推奨
- `max_attempts_per_molecule = 3000-5000` で十分な試行回数を確保